In [71]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [72]:
import graphviz
from graphviz import Graph

In [73]:
def split(df, stratify_by=None):
    """
    Crude train, validate, test split
    To stratify, send in a column name for the stratify_by argument
    """

    if stratify_by == None:
        train, test = train_test_split(df, test_size=.2, random_state=123)
        train, validate = train_test_split(train, test_size=.3, random_state=123)
    else:
        train, test = train_test_split(df, test_size=.2, random_state=123, stratify=df[stratify_by])
        train, validate = train_test_split(train, test_size=.3, random_state=123, stratify=train[stratify_by])

    return train, validate, test

In [95]:
df = pd.read_csv('titanic_df.csv')

In [96]:
df.head()

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [97]:
dummy1_df = pd.get_dummies(df[['embark_town']],drop_first=True)
dummy2_df = pd.get_dummies(df[['class']],drop_first=True)
df["is_female"] = df.sex == "Female"

In [98]:
df = pd.concat([df,dummy1_df,dummy2_df], axis=1)

df = df.drop(columns=['Unnamed: 0','sex', 'pclass','class', 'embarked','embark_town','deck'])


In [101]:
df.age.dropna(inplace=True)

In [104]:
df.show_doc()

AttributeError: 'DataFrame' object has no attribute 'show_doc'

In [78]:
train, validate, test = split(df, stratify_by="survived")

In [79]:
train.shape, validate.shape, test.shape

((498, 12), (214, 12), (179, 12))

In [80]:
# Setup our X inputs and y target variable for each split
X_train = train.drop(columns=['survived'])
y_train = train.survived # labeled data == supervise algorithm

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

In [81]:
df.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

## Exercise 1. 
### What is your baseline prediction? What is your baseline accuracy? 


In [82]:
train["most_frequent"] = 0
baseline_accuracy = (train.survived == train.most_frequent).mean()
print(f'My baseline prediction is survived = 0')
print(f'My baseline accuracy is: {baseline_accuracy:.2%}')

My baseline prediction is survived = 0
My baseline accuracy is: 61.65%


## Exercise 2.
### Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [83]:
clf = DecisionTreeClassifier(max_depth=2, random_state=123)

In [84]:
# fitting == training the model
clf = clf.fit(X_train, y_train)
clf

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').